## Домашнее задание к теме "Продвинутый pandas"

### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type

In [100]:
import pandas as pd

In [101]:
visits = pd.read_csv('pandas_data_3/visit_log.csv', sep=';')

def get_source_type(row):
    if row.traffic_source in ['yandex', 'google']:
        return 'organic'
    if row.traffic_source in ['paid', 'email']:
        if row.region == 'Russia':
            return 'ad'
        else:
            return 'other'
    return row.traffic_source

visits['source_type'] = visits.apply(get_source_type, axis=1)

visits.head(5)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей.

In [102]:
urls = pd.read_csv('pandas_data_3/URLs.txt')

urls[urls.url.str.contains('/\d{8}-', regex=True)].head(5)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [103]:
from datetime import timedelta

def calc_lifetime(row):
    return row['max'] - row['min']

ratings = pd.read_csv('pandas_data_3/ml-latest-small/ratings.csv', sep=',')
active_users = ratings.groupby('userId').timestamp.agg(['min', 'max', 'count']).query('count > 100')
active_users['lifetime'] = active_users.apply(calc_lifetime, axis=1)

avg_lifetime_in_days = timedelta(seconds=active_users.lifetime.mean()).days
print(f'Среднее время жизни {avg_lifetime_in_days} дней')

Среднее время жизни 463 дней


## Задание 4
Дана статистика услуг перевозок клиентов компании по типам. Необходимо сформировать две таблицы:
1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
1. аналогичную таблицу по типам выручки с указанием адреса клиента  

*В процессе объединения таблиц данные не должны теряться.*

In [104]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [105]:
(client_base['client_id'].to_frame() 
    .merge(rzd, on='client_id', how='left') 
    .merge(auto, on='client_id', how='left') 
    .merge(air, on='client_id', how='left') 
    .fillna(0)
)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [106]:
(client_base
    .merge(rzd, on='client_id', how='left') 
    .merge(auto, on='client_id', how='left') 
    .merge(air, on='client_id', how='left') 
    .fillna(0)
)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0
